# PCA - Esempi di applicazione

In [ ]:
# ***** NOTA BENE! *****
# perché %matplotlib widget funzioni, installare nell'ambiente virtuale 
# il pacchetto ipympl con il comando:
# pip install ipympl
#
# ATTENZIONE: perché funzioni è necessario chiudere e rilanciare jupyter-lab
#
# STILE DI VISUALIZZAZIONE PLOT FATTI CON MATPLOTLIB
# %matplotlib widget
#
#
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

# Importazione di due datasets messi a disposizione da Scikit-Learn

Guardare https://scikit-learn.org/stable/modules/classes.html?highlight=datasets#module-sklearn.datasets per maggiori informazioni sui datasets "giocattolo" messi a disposizione da Scikit-Learn.

Noi lavoreremo con il dataset "iris" ed il dataset "wine".

**Compito per lo studente:** guardare la documentazione di *datasets.load_iris* e *datasets.load_wine* (link sopra indicato) per capirne il contenuto.

In [ ]:
iris_dataset = datasets.load_iris(as_frame=True)
wine_dataset = datasets.load_wine(as_frame=True)

iris = pd.concat([iris_dataset['data'], iris_dataset['target']], axis=1)
wine = pd.concat([wine_dataset['data'], wine_dataset['target']], axis=1)

display(iris)
print(iris_dataset['DESCR'])
display(wine)
print(wine_dataset['DESCR'])

### Osservazioni:
1. Il dataset degli iris vede le feature dei fiori espresse tutte in cm e con range di valori tutti dello stesso ordine di grandezza
2. Il dataset dei vini vede features di vario tipo, espresse con unità di misura differenti e range di valori di ordine di grandezza differenti.

**Conseguenze:** 
1. Per gli iris possiamo usare la PCA "normale";
2. Per i vini non possiamo usare la PCA "normale", altrimenti la maggior parte della varianza verrebbe "mangiata" dalla feature "Proline". Per questo dataset applicheremo quindi una standardizzazione dei dati, oltre che un centramento.

Mostriamo la percentuale di varianza spiegata dalle varie PC per i due dataset. Nel caso dei vini mostriamo sia il caso con che senza standardizzazione.

**N.B.:** Per questa indagine preliminare, non ridurremo la dimensionalità del problema e considereremo quindi tutte le PC.

In [ ]:
X_iris = iris.iloc[:, :-1]  # Escludo l'ultima colonna dei target
X_wine = wine.iloc[:, :-1]  # Escludo l'ultima colonna dei target

scaler_wine = StandardScaler()
scaler_wine.fit(X_wine.values)

X_wine_scaled = scaler_wine.transform(X_wine.values)

pca_iris = PCA()
pca_wine = PCA()
pca_wine_nostd = PCA()

pca_iris.fit(X_iris.values)
pca_wine.fit(X_wine_scaled)
pca_wine_nostd.fit(X_wine.values)

# NOTE:
# np.cumsum: funzione numpy che esegue la somma cumulative del vettore in argomento
# np.insert: permette di inserire un valore in un vettore alla posizione specificata
#
# COMPITO PER LO STUDENTE: guardare la documentazione numpy per comprendere il 
# funzionamento delle funzioni sopra indicate e delle funzioni di matplotlib usate 
# nel seguito.

plt.figure()
plt.plot(np.insert(np.cumsum(pca_iris.explained_variance_ratio_), 0, 0))
plt.title('IRIS')
plt.xticks(ticks=np.arange(1, pca_iris.n_features_ + 1), 
           labels=[f'PC{i}' for i in range(1, pca_iris.n_features_ + 1)])
plt.xlabel('Principal components')
plt.ylabel('Cumulative explained variance')
plt.grid()
plt.show()

plt.figure()
plt.plot(np.insert(np.cumsum(pca_wine_nostd.explained_variance_ratio_), 0, 0))
plt.title('WINE (NO STANDARDIZATION)')
plt.xticks(ticks=np.arange(1, pca_wine_nostd.n_features_ + 1), 
           labels=[f'PC{i}' for i in range(1, pca_wine_nostd.n_features_ + 1)])
plt.xlabel('Principal components')
plt.ylabel('Cumulative explained variance')
plt.grid()
plt.show()

plt.figure()
plt.plot(np.insert(np.cumsum(pca_wine.explained_variance_ratio_), 0, 0))
plt.title('WINE (WITH STANDARDIZATION)')
plt.xticks(ticks=np.arange(1, pca_wine.n_features_ + 1), 
           labels=[f'PC{i}' for i in range(1, pca_wine.n_features_ + 1)])
plt.xlabel('Principal components')
plt.ylabel('Cumulative explained variance')
plt.grid()
plt.show()


## Visualizzazioni

Per le visualizzazioni, per esercizio sceglieremo m=2 PC per gli iris e m=3 PC per wine. Si osserva tuttavia che la varianza spiegata è molto alta per gli iris (quasi 100%) e "accettabile" per wine (quasi 70%).

In [ ]:
pca_iris_m = PCA(n_components=2)
pca_wine_m = PCA(n_components=3)

pca_iris_m.fit(X_iris.values)
pca_wine_m.fit(X_wine_scaled)

Y_iris_m = pca_iris_m.transform(X_iris.values)
Y_wine_m = pca_wine_m.transform(X_wine_scaled)

### Score Graph

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(Y_iris_m[:, 0], Y_iris_m[:, 1], c=iris['target'].values)
plt.title('IRIS - SCORE GRAPH')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid()
plt.show()

# NOTA: per i plot 3D, importare sempre Axes3D, cioè eseguire prima il 
# comando "from mpl_toolkits.mplot3d import Axes3D".
# Per ulteriori informazioni su plot 3D, consultare la documentazione di Matplotlib

fig_winescore = plt.figure()
ax = fig_winescore.add_subplot(111, projection='3d')
ax.scatter(Y_wine_m[:, 0], Y_wine_m[:, 1], Y_wine_m[:, 2], c=wine['target'].values)
plt.title('WINE - SCORE GRAPH')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.grid()
plt.show()

### Loading Graph

In [ ]:
plt.figure()
for i in range(pca_iris_m.n_features_):
    plt.plot([0, pca_iris_m.components_[0, i]], [0, pca_iris_m.components_[1, i]], 
             label=X_iris.columns[i])
plt.scatter(pca_iris_m.components_[0, :], pca_iris_m.components_[1, :], c='k')
plt.legend()
plt.title('IRIS - LOADING GRAPH')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid()
plt.show()

# NOTA: per i plot 3D, importare sempre Axes3D, cioè eseguire prima il 
# comando "from mpl_toolkits.mplot3d import Axes3D".
# Per ulteriori informazioni su plot 3D, consultare la documentazione di Matplotlib

fig_winescore = plt.figure()
ax = fig_winescore.add_subplot(111, projection='3d')
for i in range(pca_wine_m.n_features_):
    ax.plot([0, pca_wine_m.components_[0, i]], [0, pca_wine_m.components_[1, i]], 
            [0, pca_wine_m.components_[2, i]],
             label=X_wine.columns[i])
ax.scatter(pca_wine_m.components_[0, :], pca_wine_m.components_[1, :], pca_wine_m.components_[2, :], c='k')
plt.legend(bbox_to_anchor=(1.05, 1), fontsize='xx-small')
plt.title('WINE - LOADING GRAPH')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.grid()
plt.show()


### Biplot

**Compito per lo studente:** combinare i codici degli score e loading graphs per rappresentare i biplot.

In [ ]:
# ...

## Interpretazione delle PC

Vediamo il contributo delle features originali per le PC, aiutandoci con dei barplot.

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(np.arange(pca_iris_m.n_features_), pca_iris_m.components_[0, :])
plt.xticks(ticks=np.arange(pca_iris_m.n_features_), 
           labels=X_iris.columns.to_list(),
           rotation=45)
plt.title('IRIS - PC1')
plt.grid()
plt.show()

plt.figure(figsize=(8, 6))
plt.bar(np.arange(pca_iris_m.n_features_), pca_iris_m.components_[1, :])
plt.xticks(ticks=np.arange(pca_iris_m.n_features_), 
           labels=X_iris.columns.to_list(),
           rotation=45)
plt.title('IRIS - PC2')
plt.grid()
plt.show()


**Assegnare nomi alle PC:** guardando ai barplot, potremmo (*per esempio*) assegnare alla PC1 il nome di "petal size and sepal length", mentre alla PC2 il nome di "sepal size".

**ESERCIZIO PER LO STUDENTE:** stampare i barplot delle PC corrispondenti al dataset wine e guardare se è possibile "assegnare loro un nome".

In [ ]:
# ...